обьеденить создание таблицы и интервалов
сделать так чтобы lqclogdata возвращала обект curve и мы его сразу записали. все юниты и пр должно быть проверено в функции
сделать сплайс
сделать создание отдельных ласов по интервалам

In [613]:
def imports_dicts():

    with open('famdict.json', 'r', encoding='utf-8') as f:
        famdict = json.load(f)
    with open('lqcdict.json', 'r', encoding='utf-8') as f:
        lqcdict = json.load(f)

    return famdict, lqcdict

In [614]:
def cname_to_lqcname(cname, lqcdict):
    lqcname = ''
    if any(cname in sublist for sublist in list(lqcdict.values())):  # есть ли кривая в словаре lqcdict гделибо
        lqcname = list(lqcdict.keys())[list(lqcdict.values()).index(
            [sublist for sublist in list(lqcdict.values()) if cname in sublist][0])]  # выбираем из словаря ключ соотвествующий найденному в значениях имени кривой
    return lqcname

In [615]:
def get_folder_path_and_splice_state():
    root = tk.Tk()
    root.geometry("400x150")
    root.title("Folder path and Splice state form")

    folder_path = ""

    def select_folder_path():
        nonlocal folder_path
        folder_path = filedialog.askdirectory()
        root.destroy()

    def get_splice_state_and_close():
        splice_state = splice_checkbutton_var.get()
        root.destroy()

    folder_path_label = tk.Label(root, text="Select Folder:")
    folder_path_label.pack()
    folder_path_entry = tk.Entry(root, width=50)
    folder_path_entry.pack()

    select_folder_path_button = tk.Button(root, text="Browse", command=select_folder_path)
    select_folder_path_button.pack()

    splice_checkbutton_var = tk.IntVar()
    splice_checkbutton = tk.Checkbutton(root, text="Splice", variable=splice_checkbutton_var)
    splice_checkbutton.pack()

    ok_button = tk.Button(root, text="Ok", command=get_splice_state_and_close)
    ok_button.pack()

    root.mainloop()

    return folder_path, splice_checkbutton_var.get()

In [616]:
def make_las_project(las_path):
    las_list = []
    for root, dirs, files in os.walk(las_path):
        for file in files:
            if file.lower().endswith(".las"):
                las_list.append(os.path.join(root, file))

    las_project = Project.from_las(las_list, index='M')

    return las_project

In [617]:
def make_intevals(las_project, classification_distance=300):
    dataset_list = {}
    for ind, w in enumerate(las_project):

        well_name = str(w.name).replace(" ", "")
        ds_start = w.header[w.header['mnemonic'] == 'STRT']['value'].values[0]
        ds_stop = w.header[w.header['mnemonic'] == 'STOP']['value'].values[0]
        if ds_start > ds_stop:
            ds_start, ds_stop = ds_stop, ds_start

        if well_name == 'WELL' or well_name == '':
            well_name = str(w.fname)
            print("!!! file {} does not contain well name. Using file name instead".format(w.fname)) #задавать вручную

        if well_name not in dataset_list.keys() :
            dataset_list[well_name] = []

        dataset_list[well_name].append([well_name, ind, ds_start, ds_stop, ds_stop - ds_start])


    for well_name in dataset_list.keys():
        ds_df = pd.DataFrame(dataset_list[well_name], columns=['well_name', 'project_index', 'top', 'bottom', 'size'])
        ds_df.sort_values(by=['top', 'size'], inplace=True)
        dbs = DBSCAN(eps=classification_distance, min_samples=1).fit(ds_df[['top', 'bottom']])
        ds_df['labels'] = dbs.labels_
        ds_df = ds_df.drop('size', axis=1)
        #dataset_list_out[well_name] = ds_df.to_dict('split')['data']

    #print(dataset_list_out)
    return ds_df

In [618]:
def make_table(lasproject, dataset_list, lqcdict):
    columns =['Well Name']
    # Create a new workbook and worksheet for the output

    for i, log in enumerate(lqcdict.keys()):
        columns.append(log)
    columns.append('Other Logs')
    logsdata = pd.DataFrame(columns=columns)

    # Write the header row
    for  i, well in enumerate(lasproject):
        #print(well)
        # Get the well name
        well_name = well.name
        ds_label=dataset_list.loc[dataset_list['project_index']==i]['labels'].values[0]


        if well_name not in logsdata['Well Name'].values:
            logsdata = logsdata.append(pd.Series(name=well_name))
            logsdata.loc[well_name, 'Well Name'] = well_name

        logsdata.fillna('', inplace=True)


        # Loop through all the logs in the well object
        for log_name in well.data.keys():

            #start and stop depth of the log
            w = well.data[log_name]

            start_depth = well.data[log_name].start
            stop_depth = well.data[log_name].stop
            lqcname = cname_to_lqcname(log_name, lqcdict)
            #check if log_name exists in the datafarme columns
            if lqcname!='' and lqcname in logsdata.columns:


                if logsdata.loc[well_name, lqcname].find(':')==-1:
                    logsdata.loc[well_name, lqcname] = "{}: {} - {} \n".format(ds_label,start_depth,stop_depth)
                else:
                    ind = int(logsdata.loc[well_name, lqcname].find(':'))

                    if int(logsdata.loc[well_name, lqcname][0:ind]) < int(ds_label):

                        logsdata.loc[well_name, lqcname] = logsdata.loc[well_name, lqcname] +"{}: {} - {} \n".format(ds_label,start_depth,stop_depth)
                    else:
                        logsdata.loc[well_name, lqcname] = "{}: {} - {} \n".format(ds_label,start_depth,stop_depth) + logsdata.loc[well_name, lqcname]

            else:

                logsdata.loc[well_name, "Other Logs"] = logsdata.loc[well_name, "Other Logs"] + log_name +', '





    # Save the dataframe as xls file
    logsdata.to_excel('logs.xls', index=False)



In [619]:
#идея в том чтобы возвращать массив данных кривойб юниты.чтобы сразу записать в обьект welly

def lqclogdata(wLQC, dataset_ind, dslabel_):
    logs = []
    lqcname = ''
    unitname = ''
    version_index= 0
    for cname in las_project[dataset_ind].data.keys():
        if cname_to_lqcname(cname,lqcdict)!= '':
            lqcname = cname_to_lqcname(cname,lqcdict)  + dslabel_  #выбираем из словаря ключ соотвествующий найденному в значениях имени кривой
            if lqcname in wLQC.data.keys():
                clist = [i for i in wLQC.data.keys() if i.find(lqcname) != -1]
                if len(clist) >= 1:
                    lqcname = lqcname + "_" + str(len(clist))

            if lqcname.find("_") == -1:
                version_index = len(lqcname)
            else:
                version_index = lqcname.find("_")

            if lqcname[0:version_index] in famdict.keys():
                if las_project[dataset_ind].data[cname].units != famdict[lqcname[0:version_index]][1]:

                    if lqcname == "NPHI":  # добавить проверку по конкретным методам ГК НК
                        if las_project[dataset_ind].data[cname].mean().iloc[0] < 1:
                            unitname = "v/v"
                        else:
                            unitname = "%"

                    elif lqcname == "GR":
                        if las_project[dataset_ind].data[cname].mean().iloc[0] < 50:
                            unitname = "uR/H"
                        else:
                            unitname = "Gapi"

                    elif lqcname == "CALI":
                        if las_project[dataset_ind].data[cname].mean().iloc[0] < 1:
                            unitname = "m"
                        else:
                            unitname = "mm"
                    else:
                        unitname = famdict[lqcname[0:version_index]][1]
                else:
                    unitname = las_project[dataset_ind].data[cname].units

                logs.append([cname, lqcname, unitname])


    return logs

In [620]:
def make_lqc_las(dataset_list, las_project):
    dataset_list.sort_values(by=['well_name', 'labels'], inplace=True)
    print(dataset_list)
    for well in list(dataset_list["well_name"].unique()):

        #determine maximum depth for lqc dataset and create list for dataframe index

        max_depth = dataset_list.loc[dataset_list["well_name"]==well,'bottom'].max()
        min_depth = dataset_list.loc[dataset_list["well_name"]==well,'top'].min()

        #MD = [round(d * 0.1, 1) for d in range(0, int(round((max_depth + 5) / 0.1, 0)) + 1)]

        wLQC = Well()
        wLQC.name = well


        for index, row in dataset_list[dataset_list["well_name"]==well].iterrows():

            dataset_ind = row["project_index"]
            dslabel = row["labels"]

            if dslabel == 0:
                dslabel_ = ""
            else:
                dslabel_ = "_" + str(dslabel)


            #copy curve object correctly to preserve unit information
            #assign or check units for curves
            for cname, lqcname, unitname in lqclogdata(wLQC, dataset_ind, dslabel_):

                wLQC.data[lqcname] = Curve(data=las_project[dataset_ind].data[cname].values, index=las_project[dataset_ind].data[cname].index,mnemonic=lqcname, units=unitname)\
                    .to_basis(start=min_depth-5, stop=max_depth + 5, step=0.1)

        if len(wLQC.data.keys()) > 0:
            wLQC.to_las(well +"_LQC.las")

In [621]:
import json
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from welly import Well, Project, Curve
import matplotlib.pyplot as plt
import os
import tkinter as tk
from tkinter import filedialog

if __name__ == "__main__":
    famdict, lqcdict = imports_dicts()

    #las_path, LWD_splice = folder_path_and_splice_state_form()
    las_path = "/Users/victorpotysyev/PycharmProjects/lqc_maker/las_data/511"

    las_project = make_las_project(las_path)
    dataset_list = make_intevals(las_project)
    make_table(las_project, dataset_list, lqcdict)


    make_lqc_las(dataset_list, las_project)




0it [00:00, ?it/s]/opt/anaconda3/lib/python3.9/site-packages/welly/las.py:144: UserWarning: Warning, LAS version 3.0 not yet supported. Attempting to use LAS 1.2 and 2.0 parsing logic for LAS 3.0.
  datasets = from_lasio(las)
1it [00:00,  8.19it/s]/opt/anaconda3/lib/python3.9/site-packages/welly/las.py:144: UserWarning: Warning, LAS version 3.0 not yet supported. Attempting to use LAS 1.2 and 2.0 parsing logic for LAS 3.0.
  datasets = from_lasio(las)
/opt/anaconda3/lib/python3.9/site-packages/welly/las.py:144: UserWarning: Warning, LAS version 3.0 not yet supported. Attempting to use LAS 1.2 and 2.0 parsing logic for LAS 3.0.
  datasets = from_lasio(las)
3it [00:00,  6.25it/s]/opt/anaconda3/lib/python3.9/site-packages/welly/las.py:144: UserWarning: Warning, LAS version 3.0 not yet supported. Attempting to use LAS 1.2 and 2.0 parsing logic for LAS 3.0.
  datasets = from_lasio(las)
4it [00:00,  6.47it/s]/opt/anaconda3/lib/python3.9/site-packages/welly/las.py:144: UserWarning: Warning, L

0
2
0
2
2
2
2
2
2
2
0
0
1
   well_name  project_index      top   bottom  labels
3       511R              3   112.90  1250.30       0
17      511R             17   516.20  1240.30       1
16      511R             16   523.10  1257.40       1
13      511R             13   565.50  1282.05       1
8       511R              8   567.26  1252.66       1
6       511R              6  1004.82  3201.42       2
4       511R              4  1142.74  3180.54       2
9       511R              9  1153.57  3191.27       2
11      511R             11  1156.60  3190.90       2
2       511R              2  1157.04  3191.34       2
14      511R             14  1183.40  3198.00       2
10      511R             10  3159.91  3528.51       3
15      511R             15  3159.91  3528.51       3
5       511R              5  3160.60  3520.30       3
12      511R             12  3186.10  3524.50       3
0       511R              0  3188.10  3519.10       3
1       511R              1  3203.30  3520.30       3
7 